In [241]:
import models
import itertools
from copy import deepcopy
# 4 seems to be the general idea. Could be different. Need to find out
d = distortion_limit = 4

# Start working on a single sentence generalize later
sentence_orig = "honorables sénateurs , que se est - il passé ici , mardi dernier ?"
sentence = sentence_orig.split(' ')

# Get a translation for each set of words using tm
tm = models.TM("../data/tm", 1)
lm = models.LM("../data/lm")

sc_P = []

for i in range(0, len(sentence)):
    for j in range(i, len(sentence)):
        if i == j:
            subset = (sentence[i],)
        else:
            subset = tuple(sentence[i:j+1])
        try:
            tm_output = tm[subset]
            sc_P.append((i, j, tm_output[0]))
        except KeyError:
            continue
            

Reading translation model from ../data/tm...
Reading language model from ../data/lm...


In [242]:
sc_P

[(0, 0, phrase(english='honourable', logprob=0.0)),
 (1, 1, phrase(english='senators', logprob=-0.124938733876)),
 (1, 2, phrase(english='senators ,', logprob=-0.176091253757)),
 (2, 2, phrase(english=',', logprob=-0.0358480773866)),
 (2, 3, phrase(english=', that', logprob=-0.195274576545)),
 (2, 4, phrase(english=', that if', logprob=0.0)),
 (2, 7, phrase(english=', what', logprob=0.0)),
 (2, 8, phrase(english=', what happened', logprob=0.0)),
 (3, 3, phrase(english='that', logprob=-0.154812589288)),
 (3, 4, phrase(english='that if', logprob=-0.412180453539)),
 (3, 5, phrase(english='what has', logprob=-0.301030009985)),
 (3, 7, phrase(english='what', logprob=0.0)),
 (3, 8, phrase(english='what happened', logprob=-0.176091253757)),
 (4, 4, phrase(english='is', logprob=-0.628903687)),
 (4, 5, phrase(english='has', logprob=-0.39545121789)),
 (4, 6, phrase(english='did', logprob=0.0)),
 (4, 7, phrase(english='come he did', logprob=-0.301030009985)),
 (4, 8, phrase(english='has happened'

# Let's start building helper functions

In [243]:
from collections import namedtuple

In [244]:
# define bitstring as a list of zeros
bitstring = [0]*len(sentence)

# define a numedtuple which will be the state vector q
State = namedtuple('State',"e1 e2 bitstring r alpha")

# this is a container that will hold all states
state_holder = {}

# initial state
q0 = State('<s>', '<s>', bitstring, 0, 0)

# append initial state to state_holder
state_holder[0] = [q0]

def ph(q, d=4):
    ph_states = []
    for state in sc_P:
        flag = True # we assume it as a valid state
        s = state[0]
        t = state[1]

        orig_bitstring = q.bitstring
        
        '''
        Step 1: Ensure bit string is not overlapped
        '''
        if s == t:
            # invalid state
            if orig_bitstring[s] != 0:
                flag = False
        else:
            # individial bits s and t are 0, but we also have to check in between them
            if orig_bitstring[s] == 0 and orig_bitstring[t] == 0:
                for _num in range(s, t+1):
                    if orig_bitstring[_num] != 0:
                        flag = False
            else:
                flag = False
        
        '''
        Step 2: Ensure distortion limit is still obeyed
        '''
        
        r = q.r
        
        if not (abs(r + 1 - s) <= d):
            flag = False
    
        if flag == True:
            ph_states.append(state)
            
    return ph_states

In [245]:
ph(q0)

[(0, 0, phrase(english='honourable', logprob=0.0)),
 (1, 1, phrase(english='senators', logprob=-0.124938733876)),
 (1, 2, phrase(english='senators ,', logprob=-0.176091253757)),
 (2, 2, phrase(english=',', logprob=-0.0358480773866)),
 (2, 3, phrase(english=', that', logprob=-0.195274576545)),
 (2, 4, phrase(english=', that if', logprob=0.0)),
 (2, 7, phrase(english=', what', logprob=0.0)),
 (2, 8, phrase(english=', what happened', logprob=0.0)),
 (3, 3, phrase(english='that', logprob=-0.154812589288)),
 (3, 4, phrase(english='that if', logprob=-0.412180453539)),
 (3, 5, phrase(english='what has', logprob=-0.301030009985)),
 (3, 7, phrase(english='what', logprob=0.0)),
 (3, 8, phrase(english='what happened', logprob=-0.176091253757)),
 (4, 4, phrase(english='is', logprob=-0.628903687)),
 (4, 5, phrase(english='has', logprob=-0.39545121789)),
 (4, 6, phrase(english='did', logprob=0.0)),
 (4, 7, phrase(english='come he did', logprob=-0.301030009985)),
 (4, 8, phrase(english='has happened'

# The $next()$ function

In [246]:
def next(q, p, eta=-1):
    s = p[0]
    t = p[1]

    num_translated_words = len(p[2].english.split(' '))

    if num_translated_words >= 2:
        last_word = p[2].english.split(' ')[-1]
        second_last = p[2].english.split(' ')[-2]
    elif num_translated_words == 1:
        last_word = p[2].english.split(' ')[-1]
        second_last = q.e2

    # Calculate new_bitstring

    new_bitstring = deepcopy(q.bitstring)

    for i in range(s, t+1):
        # just a safeguard..check whether earlier these values were zeros or not

        if q.bitstring[i] == 1:
            print('MAJOR ERROR!')
            print('The Ph(q) function has issues if you can read this...')
        new_bitstring[i] = 1

    '''
    Calculate new logprob (alpha value)
    '''

    # CALCULATE LANGUAGE MODEL PROBABILITY

    e1 = q.e1
    e2 = q.e2
    prob = 0
    if len(p[2].english.split(' ')) == 1: # just a single word
#         print('Single english word')
        # first try bigram probability
        word = p[2].english.split(' ')[0]
#         print('\n\tFinding probability of {}, {}, {}'.format(e1, e2, word))
        try:
    #         print('Finding probability of {}, {}, {}'.format(e1, e2, word))
            # the index 1 contains the logprob, 0 index contains the repeated state
            prob += lm.score((e1, e2), word)[1]
        except KeyError:
#             print("\t\tCouldn't find trigram prob..backing off to bigram")
            try:
#                 print('\t\t\tFinding probability of {}, {}'.format(e2, word))
                prob += lm.score((e2, ), word)[1]
            except KeyError:
                prob = -5.0
                print("\t\t\t\tCould not find either trigram or bigram probability..")
    else: # there are multiple words
#         print('Multiple english words')
        for _num in range(0, len(p[2].english.split(' '))):
            word = p[2].english.split(' ')[_num]
#             print('\n\tFinding probability of {}, {}, {}'.format(e1, e2, word))
            try:
                prob += lm.score((e1, e2), word)[1]
            except KeyError:
#                 print("\t\tCouldn't find trigram prob..backing off to bigram")
                try:
#                     print('\t\t\tFinding probability of {}, {}'.format(e2, word))
                    prob += lm.score((e2, ), word)[1]
                except KeyError:
                    prob = -5.0
                    print("\t\t\t\tCould not find either trigram or bigram probability..")

            e1 = deepcopy(e2)
            e2 = deepcopy(word)

    # CALCULATE G(x) LOGPROB

    g_x = p[2].logprob

    # CALCULATE DISTORTION VALUE
    dist_val = eta * abs(q.r + 1 - s)

    # UPDATE ALPHA VALUE

    new_alpha = q.alpha + prob + dist_val

    new_state = State(e1=second_last, e2=last_word, bitstring=new_bitstring, r=p[1], alpha=new_alpha)
    return new_state

# The $eq(q1, q2)$ function

In [247]:
def eq(q1, q2):
    if q1.e1 != q2.e1:
        return False
    elif q1.e2 != q2.e2:
        return False
    elif q1.bitstring != q2.bitstring:
        return False
    elif q1.r != q2.r:
        return False
    else:
        return True

In [248]:
import numpy as np

In [250]:
def add(Q_main, index, q_new, q, valid_phrase, back_pointer):
    for q_dd in Q_main[index]: # q double dash
        if eq(q_new, q_dd) == True:
            if q_new.alpha > q_dd.alpha: # score of new thing is greater than older
                Q_main[index].remove(q_dd)
                Q_main[index].append(q_new)
                back_pointer[q_new.r] = (q.r, valid_phrase)
                return
            else:
                return
    Q_main[index].append(q_new)
    back_pointer[q_new.r] = (q.r, valid_phrase)
    return

In [251]:
def beam(Q_main, index, beam_width=5):
    running_max = 0
    
    # find the highest scoring state in the set
    for q in Q_main[index]:
        if q.alpha > running_max:
            running_max = q.alpha
            curr_max_state = q
    
    final_beam = running_max - beam_width
    
    for q in Q_main[index]:
        if q.alpha < final_beam:
            Q_main[index].remove(q)
            
    return Q_main[index]

In [252]:
sentence

['honorables',
 's\xc3\xa9nateurs',
 ',',
 'que',
 'se',
 'est',
 '-',
 'il',
 'pass\xc3\xa9',
 'ici',
 ',',
 'mardi',
 'dernier',
 '?']

In [253]:
q0 = State('<s>', '<s>', bitstring, 0, 0)

In [254]:
sc_P

[(0, 0, phrase(english='honourable', logprob=0.0)),
 (1, 1, phrase(english='senators', logprob=-0.124938733876)),
 (1, 2, phrase(english='senators ,', logprob=-0.176091253757)),
 (2, 2, phrase(english=',', logprob=-0.0358480773866)),
 (2, 3, phrase(english=', that', logprob=-0.195274576545)),
 (2, 4, phrase(english=', that if', logprob=0.0)),
 (2, 7, phrase(english=', what', logprob=0.0)),
 (2, 8, phrase(english=', what happened', logprob=0.0)),
 (3, 3, phrase(english='that', logprob=-0.154812589288)),
 (3, 4, phrase(english='that if', logprob=-0.412180453539)),
 (3, 5, phrase(english='what has', logprob=-0.301030009985)),
 (3, 7, phrase(english='what', logprob=0.0)),
 (3, 8, phrase(english='what happened', logprob=-0.176091253757)),
 (4, 4, phrase(english='is', logprob=-0.628903687)),
 (4, 5, phrase(english='has', logprob=-0.39545121789)),
 (4, 6, phrase(english='did', logprob=0.0)),
 (4, 7, phrase(english='come he did', logprob=-0.301030009985)),
 (4, 8, phrase(english='has happened'

In [255]:
ph(q0)

[(0, 0, phrase(english='honourable', logprob=0.0)),
 (1, 1, phrase(english='senators', logprob=-0.124938733876)),
 (1, 2, phrase(english='senators ,', logprob=-0.176091253757)),
 (2, 2, phrase(english=',', logprob=-0.0358480773866)),
 (2, 3, phrase(english=', that', logprob=-0.195274576545)),
 (2, 4, phrase(english=', that if', logprob=0.0)),
 (2, 7, phrase(english=', what', logprob=0.0)),
 (2, 8, phrase(english=', what happened', logprob=0.0)),
 (3, 3, phrase(english='that', logprob=-0.154812589288)),
 (3, 4, phrase(english='that if', logprob=-0.412180453539)),
 (3, 5, phrase(english='what has', logprob=-0.301030009985)),
 (3, 7, phrase(english='what', logprob=0.0)),
 (3, 8, phrase(english='what happened', logprob=-0.176091253757)),
 (4, 4, phrase(english='is', logprob=-0.628903687)),
 (4, 5, phrase(english='has', logprob=-0.39545121789)),
 (4, 6, phrase(english='did', logprob=0.0)),
 (4, 7, phrase(english='come he did', logprob=-0.301030009985)),
 (4, 8, phrase(english='has happened'

In [265]:
q0 = State('<s>', '<s>', bitstring, 0, 0)
Q_main = {k: [] for k in range(len(sentence)+1)}
Q_main[0] = [q0]
back_pointer = {}

for i in range(0, len(sentence) - 1):
    for q in beam(Q_main, i, beam_width=10):
        for valid_phrase in ph(q):
#             print('\ncurrent q: {}'.format(q))
#             print('current valid_phrase: {}\n'.format(valid_phrase))
            q_new = next(q, valid_phrase)
            index = len(np.nonzero(q_new.bitstring)[0])
            add(Q_main, index, q_new, q, valid_phrase, back_pointer)

In [266]:
Q_main[14]

[State(e1='last', e2='Tuesday', bitstring=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], r=12, alpha=-38.721474436),
 State(e1='is', e2='it', bitstring=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], r=7, alpha=-63.797439268),
 State(e1='past', e2='is', bitstring=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], r=6, alpha=-60.065237032000006),
 State(e1='it', e2='is', bitstring=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], r=6, alpha=-63.831813340000004),
 State(e1='that', e2='if', bitstring=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], r=4, alpha=-70.72856733)]

In [267]:
sentence_orig

'honorables s\xc3\xa9nateurs , que se est - il pass\xc3\xa9 ici , mardi dernier ?'

In [269]:
back_pointer

{0: (1, (0, 0, phrase(english='honourable', logprob=0.0))),
 1: (3, (1, 1, phrase(english='senators', logprob=-0.124938733876))),
 2: (0, (2, 2, phrase(english=',', logprob=-0.0358480773866))),
 3: (0, (3, 3, phrase(english='that', logprob=-0.154812589288))),
 4: (1, (4, 4, phrase(english='is', logprob=-0.628903687))),
 5: (0, (5, 5, phrase(english='is', logprob=-0.0662267655134))),
 6: (1, (6, 6, phrase(english='-', logprob=-0.0898769646883))),
 7: (2, (7, 7, phrase(english='it', logprob=-0.396981835365))),
 8: (3, (8, 8, phrase(english='past', logprob=-0.236418694258))),
 9: (4, (9, 9, phrase(english='here', logprob=-0.179900884628))),
 10: (5, (10, 10, phrase(english=',', logprob=-0.0358480773866))),
 11: (6, (11, 11, phrase(english='Tuesday', logprob=-0.00599036365747))),
 12: (10, (12, 12, phrase(english='last', logprob=-0.129672244191))),
 13: (9, (13, 13, phrase(english='?', logprob=-0.000198262714548)))}

[]

# Tester

In [54]:
len(sentence)

14

In [57]:
q = State('<s>', '<s>', [1,1,1,0,0,0,0,0,0,0,0,0,0,0], 0, 0)
ph_states = []
for state in sc_P:
    flag = True # we assume it as a valid state
    s = state[0]
    t = state[1]

    orig_bitstring = q.bitstring

    '''
    Step 1: Ensure bit string is not overlapped
    '''
    if s == t:
        # invalid state
        if orig_bitstring[s] != 0:
            flag = False
    else:
        # individial bits s and t are 0, but we also have to check in between them
        if orig_bitstring[s] == 0 and orig_bitstring[t] == 0:
            for _num in range(s, t+1):
                if orig_bitstring[_num] != 0:
                    flag = False
        else:
            flag = False

    '''
    Step 2: Ensure distortion limit is still obeyed
    '''

    r = q.r

    if not (abs(r + 1 - s) <= d):
        flag = False

    if flag == True:
        ph_states.append(state)
        
print(ph_states)

[(3, 3, phrase(english='that', logprob=-0.154812589288)), (3, 4, phrase(english='that if', logprob=-0.412180453539)), (3, 5, phrase(english='what has', logprob=-0.301030009985)), (3, 7, phrase(english='what', logprob=0.0)), (4, 4, phrase(english='is', logprob=-0.628903687)), (4, 5, phrase(english='has', logprob=-0.39545121789)), (4, 6, phrase(english='did', logprob=0.0)), (4, 7, phrase(english='come he did', logprob=-0.301030009985)), (5, 5, phrase(english='is', logprob=-0.0662267655134)), (5, 6, phrase(english='is', logprob=-0.307978868484)), (5, 7, phrase(english='is it', logprob=-0.219359949231))]


In [148]:
q = q0
p = ph_states[7]

In [149]:
q

State(e1='<s>', e2='<s>', bitstring=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], r=0, alpha=0)

In [150]:
p

(4, 7, phrase(english='come he did', logprob=-0.301030009985))

In [ ]:
def next(q, p, eta=-1):
    s = p[0]
    t = p[1]

    num_translated_words = len(p[2].english.split(' '))

    if num_translated_words >= 2:
        last_word = p[2].english.split(' ')[-1]
        second_last = p[2].english.split(' ')[-2]
    elif num_translated_words == 1:
        last_word = p[2].english.split(' ')[-1]
        second_last = q.e2

    # Calculate new_bitstring

    new_bitstring = deepcopy(q.bitstring)

    for i in range(s, t+1):
        # just a safeguard..check whether earlier these values were zeros or not

        if q.bitstring[i] == 1:
            print('MAJOR ERROR!')
            print('The Ph(q) function has issues if you can read this...')
        new_bitstring[i] = 1

    '''
    Calculate new logprob (alpha value)
    '''

    # CALCULATE LANGUAGE MODEL PROBABILITY

    e1 = q.e1
    e2 = q.e2
    prob = 0
    if len(p[2].english.split(' ')) == 1: # just a single word
        # first try bigram probability
        word = p[2].english.split(' ')[0]
        try:
    #         print('Finding probability of {}, {}, {}'.format(e1, e2, word))
            # the index 1 contains the logprob, 0 index contains the repeated state
            prob += lm.score((e1, e2), word)[1]
        except KeyError:
            print("Couldn't find trigram prob..backing off to bigram")
            try:
    #             print('Finding probability of {}, {}'.format(e2, word))
                prob += lm.score((e2, ), word)[1]
            except KeyError:
                prob = -5.0
                print("Could not find either trigram or bigram probability..")
    else: # there are multiple words
        print('Multiple english words')
        for _num in range(0, len(p[2].english.split(' '))):
            word = p[2].english.split(' ')[_num]
            print('\nFinding probability of {}, {}, {}'.format(e1, e2, word))
            try:
                prob += lm.score((e1, e2), word)[1]
            except KeyError:
                print("Couldn't find trigram prob..backing off to bigram")
                try:
                    print('Finding probability of {}, {}'.format(e2, word))
                    prob += lm.score((e2, ), word)[1]
                except KeyError:
                    prob = -5.0
                    print("Could not find either trigram or bigram probability..")

            e1 = deepcopy(e2)
            e2 = deepcopy(word)

    # CALCULATE G(x) LOGPROB

    g_x = p[2].logprob

    # CALCULATE DISTORTION VALUE
    dist_val = eta * abs(q.r + 1 - s)

    # UPDATE ALPHA VALUE

    new_alpha = q.alpha + prob + dist_val

    new_state = State(e1=second_last, e2=last_word, bitstring=new_bitstring, r=p[1], alpha=new_alpha)
    return new_state

In [156]:
q

State(e1='<s>', e2='<s>', bitstring=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], r=0, alpha=0)

In [157]:
p

(4, 7, phrase(english='come he did', logprob=-0.301030009985))

In [158]:
new_state

State(e1='he', e2='did', bitstring=[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], r=7, alpha=-11.327959700000001)

In [172]:
q1 = State('<s>', '<s>', bitstring, 0, 0)
q2 = State('<s>', '<s>', bitstring, 0, 0)

In [173]:
def eq(q1, q2):
    if q1.e1 != q2.e1:
        return False
    elif q1.e2 != q2.e2:
        return False
    elif q1.bitstring != q2.bitstring:
        return False
    elif q1.r != q2.r:
        return False
    else:
        return True

In [174]:
eq(q1, q2)

True